In [2]:
import glob
import csv
from PIL import Image
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Task 1-2

Разделение загруженного набора данных на обучающую, тестовую и валидационую выборки

In [3]:
val_path = os.path.abspath('val_list')
test_path = os.path.abspath('test_list')
train_path = os.path.abspath('train_list')

# Read and preprocess the CSV file
df = pd.read_csv('dataset3/annotation3.csv', sep=',', header=None)
df = df.drop(df.index[0])
df.drop(0, axis=1, inplace=True)
df.rename(columns={1: 'Absolute_path', 2: 'Class'}, inplace=True)
df.reset_index(inplace=True)
print(df)

# Create directories if they do not exist
for path in [val_path, test_path, train_path]:
  if not os.path.isdir(path):
   os.mkdir(path)

# Function to load and save images
def load_image(df, path, i):
  image_path = os.path.abspath(os.path.join(*df.Absolute_path[i].split("\\")))
  image = cv2.imread(image_path)
  cv2.imwrite(os.path.join(path, f'{i}.jpg'), image)

# Process images based on ranges
for i in range(900, 1000):
  load_image(df, val_path, i)
for i in range(1900, 1999):
  load_image(df, val_path, i)
for i in range(800, 900):
  load_image(df, test_path, i)
for i in range(1800, 1900):
  load_image(df, test_path, i)
for i in range(800):
  load_image(df, train_path, i)

# Create lists of image files and split the training list
train_list = glob.glob(os.path.join(train_path, '*.jpg'))
test_list = glob.glob(os.path.join(test_path, '*.jpg'))

train_list, val_list = train_test_split(train_list, test_size=0.1)


FileNotFoundError: [Errno 2] No such file or directory: 'annotation3.csv'

Проверка списков

In [140]:
print(len(images_list))
print(images_list[:5])
    
print(len(train_list))
print(train_list[:5])
    
print(len(test_list))
print(test_list[:5])
    
print(len(val_list))
print(val_list[:5])

0
[]
0
[]
0
[]
0
[]


Проверка картинок

In [138]:
random_idx = np.random.randint(1,len(images_list),size=10)

fig = plt.figure()
i=1
for idx in random_idx:
    ax = fig.add_subplot(2,5,i)
    img = Image.open(images_list[idx])
    plt.imshow(img)
    i+=1
    plt.axis('off')


plt.show()

ValueError: low >= high

Класс dataset, который будет хранить загруженные и преобразованные изображения и метки классов

In [112]:
class dataset(torch.utils.data.Dataset):
    
    def __init__(self,file_list,transform=None):
        self.file_list = file_list
        self.transform = transform
        
        
    #dataset length
    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength
    
    #load an one of images
    def __getitem__(self,idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        
        label = img_path.split('/')[-1].split('.')[1]
        if label == 'cats':
            label=0
        elif label == 'dogs':
            label=1
       
            
        return img_transformed,label

# Task 4

Пайплайн предобработки данных

In [113]:
train_transforms =  transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])

val_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])

test_transforms = transforms.Compose([   
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])
    
train_data = dataset(train_list, transform=train_transforms)
test_data = dataset(test_list, transform=test_transforms)
val_data = dataset(val_list, transform=val_transforms)

In [114]:
print(train_data[1])
print(val_data[0][1])

(tensor([[[0.6863, 0.6824, 0.6784,  ..., 0.5294, 0.5333, 0.5412],
         [0.6784, 0.6784, 0.6745,  ..., 0.5255, 0.5294, 0.5333],
         [0.6706, 0.6706, 0.6706,  ..., 0.5059, 0.5059, 0.5059],
         ...,
         [0.7490, 0.7451, 0.7451,  ..., 0.6431, 0.6392, 0.6431],
         [0.7373, 0.7373, 0.7373,  ..., 0.6353, 0.6353, 0.6392],
         [0.7216, 0.7216, 0.7216,  ..., 0.6353, 0.6353, 0.6392]],

        [[0.7961, 0.7961, 0.7961,  ..., 0.3647, 0.3725, 0.3804],
         [0.7882, 0.7922, 0.7922,  ..., 0.3686, 0.3686, 0.3725],
         [0.7804, 0.7843, 0.7882,  ..., 0.3490, 0.3490, 0.3490],
         ...,
         [0.8471, 0.8431, 0.8431,  ..., 0.7373, 0.7373, 0.7373],
         [0.8353, 0.8353, 0.8353,  ..., 0.7333, 0.7333, 0.7373],
         [0.8196, 0.8196, 0.8196,  ..., 0.7373, 0.7373, 0.7412]],

        [[0.9059, 0.9020, 0.8980,  ..., 0.2706, 0.2745, 0.2824],
         [0.9020, 0.8980, 0.8980,  ..., 0.2706, 0.2706, 0.2745],
         [0.8980, 0.8941, 0.8941,  ..., 0.2510, 0.2510, 0

# Task 3


In [94]:
lr = 0.001 # learning_rate
batch_size = 100 # we will use mini-batch method

загрузка набора данных в dataloader

In [115]:
train_loader = torch.utils.data.DataLoader(dataset = train_data, 
                                           batch_size=batch_size, 
                                           shuffle=True )

test_loader = torch.utils.data.DataLoader(dataset = test_data, 
                                          batch_size=batch_size, 
                                          shuffle=False)

val_loader = torch.utils.data.DataLoader(dataset = val_data, 
                                         batch_size=batch_size, 
                                         shuffle=False)

In [116]:
print(len(train_data), len(train_loader))
print(len(val_data), len(val_loader))
train_data[0][0].shape

1600 16
200 2


torch.Size([3, 224, 224])

Модель сверточной нейросети для решения задачи классификации

In [117]:
class Cnn(nn.Module):
    def __init__(self):
        super(Cnn,self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=3, padding=0,stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
            )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        
        self.fc1 = nn.Linear(3*3*64,10)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(10,1)
        self.relu = nn.ReLU()
        
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = torch.nn.Flatten()(out)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return torch.nn.Sigmoid()(out)

# Task 5


In [118]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1234)
if device =='cuda':
    torch.cuda.manual_seed_all(1234)

print(device)
    
model = Cnn().to(device)

cpu


In [119]:
optimizer = optim.Adam(params = model.parameters(),lr=0.001)
criterion = nn.BCELoss()

Цикл обучения нейросети

In [133]:
print('Learning rate: 0.001, Batch size: 10')

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size=10, shuffle=True )
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size=10, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset = val_data, batch_size=10, shuffle=True)

epochs = 10
def train_loop (train_loader, val_loader, epochs):
    val_loss_list = []
    val_accuracy_list = []

    loss_list = []
    accuracy_list = []

    for epoch in range(epochs):
        epoch_loss = 0
        epoch_accuracy = 0

        for data, label in train_loader:
            data = data.to(device)
            label = label.to(device)

            output = model(data)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            acc = ((output.argmax(dim=1) == label).float().mean())
            epoch_accuracy += acc/len(train_loader)
            epoch_loss += loss/len(train_loader)

        loss_list.append(epoch_loss.item())
        accuracy_list.append(epoch_accuracy.item())

        print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch+1, epoch_accuracy,epoch_loss))


        with torch.no_grad():
            epoch_val_accuracy=0
            epoch_val_loss =0

            for data, label in val_loader:
                data = data.to(device)
                label = label.to(device)

                val_output = model(data)
                val_loss = criterion(val_output,label)

                acc = ((val_output.argmax(dim=1) == label).float().mean())
                epoch_val_accuracy += acc/ len(val_loader)
                epoch_val_loss += val_loss/ len(val_loader)


            val_loss_list.append(epoch_val_loss.item())
            val_accuracy_list.append(epoch_val_accuracy.item())

            print('Epoch : {}, val_accuracy : {}, val_loss : {}'.format(epoch+1, epoch_val_accuracy,epoch_val_loss))
        num_epochs = [i+1 for i in range(epochs)]

Learning rate: 0.001, Batch size: 10


In [135]:
train_loop (train_loader, val_loader, epochs)

AttributeError: 'tuple' object has no attribute 'to'